# LangGraph

In [1]:
# Build a simple joke bot:
# The agent greets the user: “Hey, wanna hear a joke?”
# The user replies. The agent checks if they want a joke or not.
# If yes, the agent asks what kind of joke they want.
# The agent checks if the user’s preference is valid; if valid, it tells a joke, otherwise it prompts the user again.
# After telling a joke, the agent asks if the user wants another joke. This continues until the user says no, which ends the conversation.

LangGraph setup

# Libraries

In [2]:
# Set OpenAI key in the environment
from google.colab import userdata
import os

api_key = userdata.get('genai_course')
os.environ['OPENAI_API_KEY'] = api_key

In [3]:
!pip install -q langchain-community langchain-openai langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [4]:
!pip install -q grandalf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.2 MB/s eta 0:00:00


In [5]:
from openai import OpenAI

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.graph import START, StateGraph, END

from IPython.display import Markdown, Image
from typing_extensions import TypedDict

import json
import ast

In [6]:
# create openAI client
client = OpenAI()

# TypedDict

Helps with dictionaries.
It helps keep track wich data labels, these will be keys go with wich data types.

A typeddict maps data labels to specific data types, just like a dictionary blueprint with set fields and types.

When we define a TypeDict, lets say name AgentState, python knows exactly what kind of data we want.

Example:
```
class AgentState(TypedDict):
  topic: str
  topic_validity: str
  response: str
  cont: str
  recursion_limit: int
```

1. Topic - to store the type of joke
2. topic_validity - to keep track of the topic validity
3. response - to store user responses by step
4. cont - an extra field you might to pass along
5. recursion_limit - to cap conversation runs

## Why use TypedDict?

1. Reminds what fields dictionary should we have

2. Helps to see what each field does

3. Reduces mistakes

## Where does it fit into our Joke Bot?

In our joke bot code, AgentDtate, travels, carrying the user's data along the conversation. Each step updates the state for the workflow to use.

AgentState defines needed values up front; StateGraph runs steps and updates fields.

## Key takeaways

TypedDict is like a blueprint for dictionaries.
It helps maintain consistency.
It improves readability and reduces errors in your code.
I our program, it holds key conversation details shared between functions.

AgentState sets guardrails for data use, making code clearer and easier to maintain.






TypedDict Class

This is like a Schema or a DataFrame with a dictionary as a type data. Here the Object state is going to travel through all the graph

In [7]:
# Build a TypedDict class to store information
class AgentState(TypedDict):
  topic: str
  topic_validity: str # output of the LLM
  response: str # output from the user
  #cont: str # the continuation of the conversation
  recursion_limit: int # dont want to be running for ever lets say only 5 or ten interations

We are now going to build some functions that connect and collect data and we are going to have a class of AgentState() so it will always be updated.

# AI APP functions

## Greet User

In [8]:
def greet_user(state): # The input is always going to be state because the input of each of the functions is the last known state of the conversation
  system_message = """
  You are Chris Rock, the funny comedian,
  you ask if the user wants to hear a joke.
  Instruct to answer with a yes or no.
  """,
  completion = client.chat.completions.create(
      model = 'gpt-4.1-mini',
      messages = [
          {'role': 'system', 'content': system_message}
      ]
  )
  greet = completion.choices[0].message.content
  print(greet)
  user_input = input()
  state['response'] = user_input

  return state

Get Topic of Joke - Part 1

## Topic

1. def get_topic(state)::

* Aquí se define una función que representa un "nodo" o un paso en el flujo de tu agente.

* Lo más importante es el argumento state. En el desarrollo de agentes (como con LangGraph o flujos simples), el state suele ser un diccionario que actúa como la memoria compartida del agente. Pasa de una función a otra acumulando información.

2. state['topic'] = user_input:

* Aquí es donde guardamos lo que dijo el usuario en la "memoria" del agente.

* Se crea (o actualiza) la clave 'topic' dentro del diccionario state. Esto es crucial para que los siguientes pasos del agente sepan de qué habló el usuario.

In [9]:
# Define a function that gets and validated the topic
def get_topic(state):
  print('What type of joke are you in the mood for?')
  user_input = input()
  state['topic'] = user_input
  system_message = """
  Validate if the the user's preference is a valid type of joke.
  Return as JSON:
  If valid, return {'valid': 'yes', 'response': <a joke related to the preference>}.
  If invalid, return {'valid': 'no', 'response': 'Please enter a valid topic'}.
  """,
  completion = client.chat.completions.create(
      model = 'gpt-4o-mini',
      messages = [{'role': 'system', 'content': system_message},
                  {'role': 'user', 'content': f'{state['topic']}'}],
      response_format = {'type': 'json_object'} # It has to be on JSON becaus LLM thinks with strings not in Python
  )
  print(completion.choices[0].message.content)

  try:
    # AQUÍ ocurre la transformación: Texto (JSON) -> Diccionario (Python)
    bot = json.loads(completion.choices[0].message.content)
  except json.decoder.JSONDecodeError:
    print('Model did not return a valid JSON')

  # Ahora 'bot' YA ES un diccionario. Puedes usar .get()
  state['topic_validity'] = bot.get('valid')
  print(bot.get('response'))

  return state

In [10]:
# Nodo goodbye(state)
def goodbye(state):
  print('Goodbye!')
  return state

## Routers

In [11]:
def response_router(state):
  if state['response'].lower() == 'yes':
    return 'Continue'
  elif state['response'].lower() == 'no':
    return 'End'
  else:
    return 'Continue'

In [12]:
def topic_router(state):
  if state['topic_validity'] == 'yes':
    return 'Valid'
  elif state['topic_validity'] == 'no':
    return 'Invalid'

In [13]:
def continuation(state):
  print('Do you want another joke? Reply continue or end:')
  user_input = input()
  state['response'] = user_input
  return state

# WorkFlow

In [14]:
# Initializing a StateGraph using the AgentState
workflow = StateGraph(AgentState)

# Set an entry point for the workflow
workflow.set_entry_point('Greet User')

# Define the nodes and their functions
workflow.add_node('Greet User', greet_user)
workflow.add_node('Get Topic & Check Validity', get_topic)
workflow.add_node('Continue?', continuation)
workflow.add_node('Say Bye', goodbye)

# Define a conditional edge
workflow.add_conditional_edges(
    'Greet User', # This is were we are
    response_router,
    {'Continue': 'Get Topic & Check Validity', # Si router dice 'Continue' -> Ve a este nodo
     'End': 'Say Bye'} # # Si router dice 'End' -> Ve a este nodo
)

workflow.add_conditional_edges(
    'Get Topic & Check Validity', # this is were we are
    topic_router,
    {'Valid': 'Continue?',
     'Invalid': 'Get Topic & Check Validity'} # # <--- ¡RECURSIVIDAD! (Loop)
)

workflow.add_conditional_edges(
    'Continue?',
    response_router, # The function that we are going to apply
    {'Continue': 'Get Topic & Check Validity',
     'End': 'Say Bye'}
)

# Workflow add edge
workflow.add_edge('Say Bye', END)

# Compile the workflow
app = workflow.compile()

In [19]:
# Display the flow
display(Image(app.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink API while trying to render your graph. Status code: 404.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [17]:
# Incase you have issues with the image
app.get_graph().print_ascii()

ValueError: no intersection found (point inside ?!). view: <langchain_core.runnables.graph_ascii.VertexViewer object at 0x7bfde7dbaa50> topt: (-16.25, 13.5)

In [18]:
!pip install pyppeteer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 11.4 MB/s eta 0:00:00
  Created wheel for websockets: filename=websockets-10.4-cp312-cp312-linux_x86_64.whl size=107329 sha256=c5fe077d611780710ef3412ac5c0bbf7c89ccd0888072916d266b30ab2fad65a
  Stored in directory: /root/.cache/pip/wheels/80/cf/6d/5d7e4c920cb41925a178b2d2621889c520d648bab487b1d7fd
Successfully built websockets
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0
ERROR:

In [ ]:
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display, Image

# Asegúrate de instalar pyppeteer primero: !pip install pyppeteer

display(Image(
    app.get_graph().draw_mermaid_png(
        # 🚨 SOLUCIÓN: Usar el método de renderizado local
        draw_method=MermaidDrawMethod.PYPPETEER
    )
))

RuntimeError: asyncio.run() cannot be called from a running event loop

I keep having problems visualizing the graph :(

In [ ]:
# Run the app
# result = app.invoke({'recursion_limit': 10})